In [ ]:
https://www.kaggle.com/code/ammarkhan071/synthea-analysis

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing the data in allergies.csv

In [ ]:
allergies = pd.read_csv("../data/allergies.csv")
allergies.head()

lets see the shape of the data

In [ ]:
def isCured(x):
    if type(x) != str:
        return 0
    else:
        return 1

In [ ]:
allergies["isCured"] = allergies.STOP.apply(isCured)
allergies.head()

In [ ]:
allergies.shape

so it has 597 rows and 6 columns

Lets see how many distinct kind of allergies exists

In [ ]:
allergies.DESCRIPTION.nunique(),allergies.DESCRIPTION.count(),allergies.CODE.nunique(),allergies.CODE.count()

22 distinct allergies exist throughout the data. Lets see which are the most common ones and which are the least common ones.

In [ ]:
allergy_group = allergies.groupby(["DESCRIPTION"]).agg({"DESCRIPTION":"count","isCured":"sum"})
allergy_group = allergy_group.rename(columns = {"DESCRIPTION":"COUNT"})
allergy_group = allergy_group.reset_index().sort_values(by = ["COUNT"],ascending = False)
allergy_group["isNotCured"] = allergy_group["COUNT"] - allergy_group["isCured"]
allergy_group["RecoveryRate"] = np.round(100*allergy_group["isCured"]/allergy_group["COUNT"],0)
allergy_group.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (20,10))
plt.barh(allergy_group.DESCRIPTION,allergy_group.isCured,color = "orange")
plt.barh(allergy_group.DESCRIPTION,allergy_group.isNotCured,left = allergy_group.isCured)
plt.legend(["No. of Patients Cured","No. of Patients yet to be cured"])
plt.title("Number of patients across different allergies")
plt.xlabel("Count of Patients")
for i in range(len(allergy_group.DESCRIPTION)):
    plt.annotate(str(allergy_group.RecoveryRate.iloc[i])+" %",(allergy_group.COUNT.iloc[i]+0.5,i-0.12))

plt.show()

From the above graph its clear that **Allergy to Mould** and **Dander allergy** are the most common ones. Whereas **soya** and **Dairy product** allergy are the least common ones.
Also allergies to peanuts,fish,nut, venom and Shellfish seems to have no treatment as there hasn't been a single patient recovered from these. Our Doctors have a very high success rate with egg allergies. A record 30% of the patients with these allergies have been cured till date.

Lets see cured patients and how much time does it take to cure different allergies

In [ ]:
from datetime import datetime
def todatetime(x):
    return datetime.strptime(x,"%Y-%m-%d")

In [ ]:
cured_allergies = allergies[~(allergies["STOP"].isna())]
cured_allergies.START = cured_allergies.START.apply(todatetime)
cured_allergies.STOP = cured_allergies.STOP.apply(todatetime)
#cured_allergies.RecoveryYear = cured_allergies.STOP.year - cured_allergies.START.year
cured_allergies.head()

In [ ]:
array = []
for i in range(len(cured_allergies)):
    array.append(cured_allergies.STOP.iloc[i].year - cured_allergies.START.iloc[i].year)
    
cured_allergies["RecoveryYears"] = array

cured_allergies_group = cured_allergies.groupby(["DESCRIPTION"]).agg({"RecoveryYears":"mean"}).sort_values(by = ["RecoveryYears"],ascending = False).reset_index()
cured_allergies_group.head()

In [ ]:
plt.figure(figsize = (10,5))
plt.barh(cured_allergies_group.DESCRIPTION,cured_allergies_group.RecoveryYears)
plt.title("Recovery time across different alergies")
plt.show()

From above chart it can be concluded that all allergies take roughly around 15 - 18 years of time to be recovered.

In [ ]:
cured_allergies

Lets import the data related to encounters

In [ ]:
encounters = pd.read_csv("../data/encounters.csv")
encounters.head()

lets see the providers,top payers giving the hospital maximum revenue.

In [ ]:
providers_grouped = encounters.groupby(["PROVIDER"]).agg({"TOTAL_CLAIM_COST":"sum","PROVIDER":"count"}).sort_values(by = "TOTAL_CLAIM_COST",ascending = False).rename(columns = {"PROVIDER":"NoPatientSeen"}).reset_index()
providers_grouped

In [ ]:
providers = pd.read_csv("../data/providers.csv")
providers

In [ ]:
tt = providers.merge(providers_grouped,left_on = "Id",right_on = "PROVIDER",how = "inner").sort_values(by = "TOTAL_CLAIM_COST",ascending = False)
tt

In [ ]:
tt["cum_of_total_cost"] = tt.TOTAL_CLAIM_COST.cumsum()
tt[tt["cum_of_total_cost"] < (0.2 * tt.TOTAL_CLAIM_COST.sum())]

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(np.arange(1,len(tt) + 1),tt.TOTAL_CLAIM_COST.cumsum())
#plt.plot(tt.TOTAL_CLAIM_COST.sum() * 0.1)
plt.axhline(y=tt.TOTAL_CLAIM_COST.sum() * 0.8,color = "black",label = "80% of revenue below this line",linestyle='dashed')
plt.axvline(x=219,color = "red",label = "219 doctors to the left of this line",linestyle='dashed')
plt.title("Cumulative revenue generated from doctors")
plt.legend()
plt.show()

In [ ]:
import squarify
plt.figure(figsize = (20,15))
squarify.plot(tt["TOTAL_CLAIM_COST"].iloc[0:219],label = tt["NAME"].iloc[0:219])
plt.show()

The above picture shows the contribution of top 219 doctors to total revenue. Cleary Gaynell Strich, Gertrudis Schraden and Vern Polowski are the top doctors with most contribution.

In [ ]:
tt.ORGANIZATION.nunique()

In [ ]:
plt.figure()
plt.barh(tt.NAME.head(10),tt.TOTAL_CLAIM_COST.head(10))
plt.show()

The above diagram shows the top doctors with most revenue 

In [ ]:
tt1 = tt.groupby("CITY").agg({"TOTAL_CLAIM_COST":"sum"}).sort_values(by = ["TOTAL_CLAIM_COST"],ascending = False)
tt1["cum_sum"] = tt1.TOTAL_CLAIM_COST.cumsum()

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(np.arange(1,len(tt1) + 1),tt1.TOTAL_CLAIM_COST.cumsum())
#plt.plot(tt.TOTAL_CLAIM_COST.sum() * 0.1)
plt.axhline(y=tt1.TOTAL_CLAIM_COST.sum() * 0.8,color = "black",label = "80% of revenue below this line",linestyle='dashed')
plt.axvline(x=75,color = "red",label = "75 cities to the left of this line",linestyle='dashed')
plt.title("Cumulative revenue generated from different cities")
plt.legend()
plt.show()

Hospital has business around 319 cities but 80% of revenue is generated from 75 cities.

In [ ]:

plt.figure(figsize = (20,15))
squarify.plot(tt["TOTAL_CLAIM_COST"].iloc[0:75],label = tt["CITY"].iloc[0:75])
plt.show()

Jamaica Plain, WestCester,Melrose are the cities where the doctors with most revenue are based in

In [ ]:
tt_organisation = tt.groupby(["ORGANIZATION"]).agg({"TOTAL_CLAIM_COST":"sum"}).sort_values(by = ["TOTAL_CLAIM_COST"],ascending = False)
tt_organisation

In [ ]:
tt.ORGANIZATION.nunique()

In [ ]:
import plotly.express as px
import pandas as pd

df = providers[["LAT","LON"]]

fig = px.scatter_geo(df,lat='LAT',lon='LON')
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

All the doctors as seen above are from single state. Zoom in the graph for more details

In [ ]:
patients = pd.read_csv("../data/patients.csv")
patients.head()

In [ ]:
df = patients[["LAT","LON"]]

fig = px.scatter_geo(df,lat='LAT',lon='LON')
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

As seen above we are only attracting patients from Massachusetts. So either the service that we provide is not good enough to attract patients from other state or the other state have good healthcare structure so they dont need to travel across states.

In [ ]:
procedures = pd.read_csv("data/procedures.csv")
procedures.head()

In [ ]:
procedures.DESCRIPTION.nunique()

In [ ]:
proc_enc = procedures.merge(encounters,left_on = "ENCOUNTER",right_on = "Id")
proc_enc.head()

In [ ]:
proc_enc.DESCRIPTION_y.nunique()

In [ ]:
proc_enc.groupby("DESCRIPTION_y").agg({"TOTAL_CLAIM_COST":"sum","ENCOUNTER":"count"}).sort_values(by = ["TOTAL_CLAIM_COST"],ascending = False)

The above table shows the revenue generated as well as count of encounter across different purpose of visits

In [ ]:
proc_enc.groupby("DESCRIPTION_x").agg({"TOTAL_CLAIM_COST":"sum","ENCOUNTER":"count"}).sort_values(by = ["TOTAL_CLAIM_COST"],ascending = False)

The above table shows the revenue generated as well as count of encounter across different procedures

The above table shows the revenue generated as well as count of encounter across different purpose of visits

In [ ]:
conditions = pd.read_csv("../data/conditions.csv")
conditions.head()

In [ ]:
conditions.DESCRIPTION.nunique()

In [ ]:
encounters.head()

In [ ]:
type(encounters.START.iloc[0])

In [ ]:
def todatetime1(x):
    x = x[0:10]
    return datetime.strptime(x,"%Y-%m-%d")



In [ ]:
encounters.START = encounters.START.apply(todatetime1)
encounters.STOP = encounters.STOP.apply(todatetime1)
type(encounters.START.iloc[0])

In [ ]:
encounters = encounters.sort_values(by = ["START"])

In [ ]:
encounters.shape,encounters.PATIENT.nunique()

As seen above there are total around 53k encounters but only 1171 patients have visited the hospital in the given window. This implies that majority of the patients return to hospital after first visit. Aquiring new patients is difficult for the hospital but keeping an old patient is something we are good at.

In [ ]:
array_of_patients_visited = []
i = 0
def old_or_new(x):
    #patient_id = encounters.PATIENT.iloc[i]
    if x in array_of_patients_visited:
        #i += 1
        return 1
    else:
        array_of_patients_visited.append(x)
        #i += 1
        return 0

In [ ]:
encounters["isExistingPatient"] = encounters.PATIENT.apply(old_or_new)

In [ ]:
encounters.head()

In [ ]:
def isNewPatient(x):
    if x == 0:
        return 1
    else:
        return 0

encounters["isNewPatient"] = encounters.isExistingPatient.apply(isNewPatient)
encounters.head()

In [ ]:
on = encounters.groupby(["START"]).agg({"isExistingPatient":"sum","isNewPatient":"sum"}).reset_index()
on

In [ ]:
plt.figure(figsize = (15,8))
plt.plot(on[["START"]],on[["isExistingPatient"]],label = "Existing")
plt.plot(on[["START"]],on[["isNewPatient"]],label = "New")
plt.legend()
#plt.legend()
plt.show()

In the above chart each point represents no of patients in a day. Surprisingly we are very good at keeping the existing patients. Most of the pateints visiting have already visited us in the past. On a given day the we have seen maximum 20 existing patients and 3 new patients.

In [ ]:
def year_month(x):
    return x.strftime("%Y-%m")

on["Year"] = pd.DatetimeIndex(on["START"]).year
on["Month"] = pd.DatetimeIndex(on["START"]).month
on["YearMonth"] = on.START.apply(year_month)
on.head()

In [ ]:
on_yearMonth = on.groupby(["YearMonth"]).agg({"isExistingPatient":"sum","isNewPatient":"sum"}).reset_index()
on_yearMonth

In [ ]:
import matplotlib.dates as mdates
plt.figure(figsize = (15,8))
plt.plot(on_yearMonth["YearMonth"],on_yearMonth["isExistingPatient"],label = "Existing Patient")
plt.plot(on_yearMonth["YearMonth"],on_yearMonth["isNewPatient"],label = "New Patient")

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval = 365))
plt.legend()
plt.show()

In [ ]:
on_clean = on[on["Year"] > 1980]
on_clean.head()

Lets look at the data 1980 onwards as there are no gap in dates after 1980

In [ ]:
plt.figure(figsize = (15,8))
plt.plot(on_clean[["START"]],on_clean[["isExistingPatient"]],label = "Existing")
plt.plot(on_clean[["START"]],on_clean[["isNewPatient"]],label = "New")
plt.legend()
#plt.legend()
plt.show()

In [ ]:
on_yearMonth2 = on_clean.groupby(["YearMonth"]).agg({"isExistingPatient":"sum","isNewPatient":"sum"}).reset_index()
on_yearMonth2

In [ ]:
on_yearMonth2["5MAexisting"] = on_yearMonth2["isExistingPatient"].rolling(window = 5).mean()
on_yearMonth2["5MAnew"] = on_yearMonth2["isNewPatient"].rolling(window = 5).mean()

on_yearMonth2

In [ ]:
#import matplotlib.dates as mdates
plt.figure(figsize = (15,8))
plt.plot(on_yearMonth2["YearMonth"],on_yearMonth2["isExistingPatient"],label = "Existing Patient")
plt.plot(on_yearMonth2["YearMonth"],on_yearMonth2["isNewPatient"],label = "New Patient")

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval = 30))
plt.xticks(rotation = 90)
plt.legend()
plt.show()

We have used 5 month Moving Average to smoothen the data. We can see that we acqiuired a lot of new customer in December 1970. Also the number of visits of existing patients shot up in 1970.

In [ ]:
conditions.head()

In [ ]:
encounters.head()

In [ ]:
enc_con = encounters.merge(conditions[["ENCOUNTER","DESCRIPTION"]],how = "inner",left_on = "Id",right_on = "ENCOUNTER")
enc_con.head()

In [ ]:
enc_con_grouped = enc_con.groupby(["DESCRIPTION_y"]).agg({"START":"mean","PATIENT":"count","TOTAL_CLAIM_COST":"mean"}).sort_values(by = ["TOTAL_CLAIM_COST"],ascending = False).reset_index()
enc_con_grouped

Lets see the number of encounters in Encounters table and Condition table

In [ ]:
encounters.Id.nunique(),conditions.ENCOUNTER.nunique

There aee 53k encounters in encounter table and in conditions table only 7.6k. That means for majority of the encounters condition has not been maintained in the data records.

In [ ]:
plt.figure(figsize = (15,8))
plt.scatter(enc_con_grouped["START"],enc_con_grouped["TOTAL_CLAIM_COST"],s = enc_con_grouped["PATIENT"])
plt.xlabel("Year")
plt.ylabel("Cost")

plt.show()

looks like the data is highly skewed in the conditions table. Costs of most conditions seems to be around 130. Other interpretation tht can be drawn from this is that they dont have a conditions data homogenous throughout the timeline. Some conditions have instances well back in 1920 and later they discontinued collecting data for that condition. At the same time for some conditions they started collecting data only in the recent days.

In [ ]:
encounters.PATIENT.nunique(),conditions.PATIENT.nunique(),len(conditions)

Above we have calculated

In [ ]:
encounters.REASONDESCRIPTION.nunique()

In [ ]:
enc_con_grouped1 = enc_con.groupby(["REASONDESCRIPTION"]).agg({"START":"mean","PATIENT":"nunique","TOTAL_CLAIM_COST":"mean","Id":"count"}).sort_values(by = ["TOTAL_CLAIM_COST"],ascending = False).reset_index()
enc_con_grouped1["Revenue"] = enc_con_grouped1["TOTAL_CLAIM_COST"]* enc_con_grouped1["Id"]
enc_con_grouped1.sort_values(by = ["Revenue"],ascending = False,inplace = True)
enc_con_grouped1

In [ ]:
plt.figure(figsize = (15,7))
plt.style.use("seaborn")
plt.scatter(enc_con_grouped1["START"],enc_con_grouped1["Revenue"],c = enc_con_grouped1["Id"],cmap = "coolwarm")
plt.colorbar().set_label("No of Encounters")
plt.xlabel("Time")
plt.ylabel("Revenue")
for i in range(10):
    plt.annotate(enc_con_grouped1.REASONDESCRIPTION.iloc[i],(enc_con_grouped1["START"].iloc[i],enc_con_grouped1["Revenue"].iloc[i]))
plt.show()

From the above graph Viral Sinusitus is the condition with most revenue and most number of encounters. Also we are encountering the patients with this condition very recently as the average of encounters date is recent. This is followed by Normal Pregnancy. Looking at the broader picture top 4 condition contributes significatly high to the revenue.